# Dogs v Cats



In [1]:
%matplotlib inline

Define path to data: (It's a good idea to put it in a subdirectory of your notebooks folder, and then exclude that directory from git control by adding it to .gitignore.)

In [2]:
root_path = "data/dogscats-redux/"
weights_path = root_path + 'keith_weights_1.h5'

#path = root_path
path = root_path + "sample/"

A few basic libraries that we'll need for the initial exercises:

In [3]:
from __future__ import division,print_function

import os, json, shutil, random, math
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

We have created a file most imaginatively called 'utils.py' to store any little convenience functions we'll want to use. We will discuss these as we use them.

In [4]:
import utils; reload(utils)
from utils import plots

Using cuDNN version 5103 on context None
Mapped name None to device cuda: Tesla K80 (9F07:00:00.0)
Using Theano backend.


In [5]:
def predict_and_show(batches):
    imgs, labels = next(batches)
    preds, indexes, predicted_labels = vgg.predict(imgs)
    true_labels = [vgg.classes[idx] for idx in np.argmax(labels, axis=1)]
    plots(imgs, titles=['ok' if a == b else 'wrong' for (a,b) in zip(true_labels, predicted_labels)])

## Use our Vgg16 class to finetune a Dogs vs Cats model

In [6]:
batch_size=16

In [7]:
import vgg16; reload(vgg16)

<module 'vgg16' from 'vgg16.py'>

In [8]:
vgg = vgg16.Vgg16()
batches = vgg.get_batches(path+'train', batch_size=batch_size)
validation_batches = vgg.get_batches(path+'validation', batch_size=batch_size)

using vgg16 architecture but NO weights
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


Calling *finetune()* modifies the model such that it will be trained based on the data in the batches provided - in this case, to predict either 'dog' or 'cat'.

In [9]:
vgg.finetune(batches)

Finally, we *fit()* the parameters of the model using the training data, reporting the accuracy on the validation set after every epoch. (An *epoch* is one full pass through the training data.)

In [10]:
hist = vgg.fit(batches, validation_batches, nb_epoch=3)

Epoch 1/3
 3280/20000 [===>..........................] - ETA: 1188s - loss: 8.1416 - acc: 0.4933

KeyboardInterrupt: 

In [ ]:
def plot_train_validation_curves(title, training_series, validation_series):
    num_epochs = len(training_series)
    epochs = range(1, num_epochs + 1)
    plt.plot(epochs, validation_series, label='validation')
    plt.plot(epochs, training_series, label='train')
    plt.xlabel('Epochs')
    plt.ylabel(title)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_train_validation_curves('Accuracy', hist.history['acc'], hist.history['val_acc'])
plot_train_validation_curves('Loss', hist.history['loss'], hist.history['val_loss'])

# Try it out

In [ ]:
dummy_batches = vgg.get_batches(path+'train', batch_size=6)

In [ ]:
imgs, labels = next(dummy_batches)
plots(imgs, titles=labels)

In [ ]:
vgg.predict(imgs)

In [ ]:
predict_and_show(dummy_batches)

# Kaggle Submission

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def get_test_batches(path, batch_size):
    gen = ImageDataGenerator()
    return gen.flow_from_directory(path, target_size=(224,224), class_mode=None, shuffle=False, batch_size=batch_size)

In [ ]:
test_batches = get_test_batches(path + 'test', batch_size=8)

In [ ]:
dog_predictions = vgg.model.predict_generator(test_batches, test_batches.nb_sample)[:,1]
dog_predictions[:5]

Extract the id of each image in the test batch because Keras ImageDataGenerator will enumerate them in lexical order

In [ ]:
ids = [int(x.split("/")[1].split(".")[0]) for x in test_batches.filenames]

fill out the submission table

In [ ]:
with open(root_path + 'keith_submission_4.csv', 'w') as f:
    f.write('id,label\n')
    for i in range(len(ids)):
        f.write('{},{}\n'.format(ids[i], dog_predictions[i]))